In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pickle
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import *

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.utils.extmath import randomized_svd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [4]:
df = pd.read_csv("Project_1_dataset_01_01_2022.csv")
print('Number of data points : ', df.shape[0])
print('Number of features : ', df.shape[1])
df.head()

Number of data points :  2072
Number of features :  9


,Unnamed: 0,full_text,summary,keywords,publish_date,authors,url,leaf_label,root_label
0,590,Having made a massive impact in Saudi Arabia w...,Having made a massive impact in Saudi Arabia w...,"['singhs', 'rooting', 'cool', 'saudi', 'style'...",NaN,[],https://www.msn.com/en-in/entertainment/other/...,cricket,sports
1,388,Cricket is all about the emotional rollercoast...,"No matter which team fans hope to win, every s...","['wants', 'fans', 'finals', 'cup', 'win', 'tou...",NaN,[],https://www.prnewswire.com:443/news-releases/c...,cricket,sports
2,423,New Zealand announces back-to-back tours next ...,New Zealand announces back-to-back tours next ...,"['test', 'west', 'tour', 'zealand', 'world', '...",2021-12-20 00:00:00,[],https://www.aljazeera.com/news/2021/12/20/cric...,cricket,sports
3,563,It's not the first time cricket fans in the co...,Billed as one of the pre-tournament favourites...,"['qualify', 'afghanistan', 'cup', 'world', 'ze...",NaN,[],https://www.msn.com/en-in/news/other/t20-world...,cricket,sports
4,634,An employee works on a computer terminal again...,"REUTERS/Sivaram VBENGALURU, Oct 11 (Reuters Br...","['success', 'tech', 'startup', 'talent', 'onli...",2021-10-11 00:00:00,['Una Galani'],https://www.reuters.com/breakingviews/india-in...,cricket,sports


In [5]:
train, test = train_test_split(df[["full_text","root_label"]], test_size=0.2)

In [6]:
print('Number of data points in train data:', train.shape[0])
print('Number of data points in test data:', test.shape[0])

Number of data points in train data: 1657
Number of data points in test data: 415


In [7]:
train

,full_text,root_label
894,"Harika Dronavalli, 30, has been marking time w...",sports
1579,Mudvayne's second reunion show took place over...,climate
89,Image Source: Getty / David M. Benett\n\nOn No...,sports
165,© Provided by Extra.ie\n\nOn the family farm i...,sports
336,Charlotte wide receiver Grant DuBose catches a...,sports
...,...,...
812,Introducing Martial Arts School Listings on Bl...,sports
358,Are the San Francisco 49ers back? Only time wi...,sports
585,"Along with the Carter kids, other celebrity ch...",sports
928,La mini serie que estuvo durante varias semana...,sports


# Cleanup

In [8]:
import re
def clean(text):
    text = re.sub(r"http\S+", '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    texter = re.sub(r"[0-9]","", texter)
    texter = re.sub('[^a-zA-Z0-9\n]', ' ', texter)
    texter = re.sub('\s+',' ', texter)
    texter = texter.lower()
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    if texter == "":
        texter = ""
    return texter

In [9]:
train_clean = train.copy()

In [15]:
train_clean

,full_text,root_label
894,"Harika Dronavalli, 30, has been marking time w...",sports
1579,Mudvayne's second reunion show took place over...,climate
89,Image Source: Getty / David M. Benett\n\nOn No...,sports
165,© Provided by Extra.ie\n\nOn the family farm i...,sports
336,Charlotte wide receiver Grant DuBose catches a...,sports
...,...,...
812,Introducing Martial Arts School Listings on Bl...,sports
358,Are the San Francisco 49ers back? Only time wi...,sports
585,"Along with the Carter kids, other celebrity ch...",sports
928,La mini serie que estuvo durante varias semana...,sports


In [22]:
trail = train_clean.copy()

In [23]:
trail

894     harika dronavalli has been marking time with c...
1579    mudvayne s second reunion show took place over...
89      image source getty david m benett on nov malal...
165      provided by extra ie on the family farm in ba...
336     charlotte wide receiver grant dubose catches a...
                              ...                        
812     introducing martial arts school listings on bl...
358     are the san francisco ers back only time will ...
585     along with the carter kids other celebrity chi...
928     la mini serie que estuvo durante varias semana...
216     england confirmed it will go to australia for ...
Name: full_text, Length: 1657, dtype: object

In [16]:
train_clean = train_clean['full_text'].apply(clean)

In [24]:
train_clean[894]

'harika dronavalli has been marking time with chess accolades but the board hasn t always been kind to her until she began competing at she led a sheltered pampered life my parents took care of everything for my sister and me she says once she started competing and winning she faced an unexpected battle the pressure to win turned to stress then anxiety her parents supported her and rejoiced at every win her father sold some of the family s real estate so they could take her to spain where she won her first international medal a silver at the world youth chess championship u girls she was at she became asia s youngest woman international master or wim at she made it onto the indian national team at she won an arjuna award at she became only the second indian woman to become a grandmaster after koneru humpy by then she had won the under girls and under girls world youth chess championships and the girls world junior chess championship title amid it all she developed a fear of losing in a

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import pos_tag
from pickle import dump

counts = []

wnl = nltk.wordnet.WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(list_word):
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]


def rmv_nums(doc):
    #gets rid of numbers including floats
    #does lemmatization with nltk.wordnet.WordNetLemmatizer and pos_tag
    return (word for word in lemmatize_sent(analyzer(doc)) 
            if not word.isdigit())

#CountVectorizer returns a callable that handles preprocessing and tokenization
#Use the “english” stopwords of the CountVectorizer
vectorizer=CountVectorizer(analyzer=rmv_nums,min_df=3,stop_words='english')

#do feature extraction (train):
X_train_counts=vectorizer.fit_transform(train["full_text"]) #get matrix of doc-term counts (training data)
print('Size of training data after lemmatization but before TF-IDF: ', X_train_counts.shape) 
X_test_counts=vectorizer.transform(test["full_text"]) 
print('Size of testing data after lemmatization but before TF-IDF:  ', X_test_counts.shape) 

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Shape of train TF-IDF matrix: ',X_train_tfidf.shape)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
print('Shape of test TF-IDF matrix:  ',X_test_tfidf.shape)

Size of training data after lemmatization but before TF-IDF:  (1657, 10427)
Size of testing data after lemmatization but before TF-IDF:   (415, 10427)
Shape of train TF-IDF matrix:  (1657, 10427)
Shape of test TF-IDF matrix:   (415, 10427)


In [26]:
y_train_encoded = train["root_label"].copy()
y_test_encoded = test["root_label"].copy()

y_train_encoded[y_train_encoded == 'sports'] = 0
y_test_encoded[y_test_encoded == 'sports'] = 0


y_train_encoded[y_train_encoded== 'climate'] = 1
y_test_encoded[y_test_encoded == 'climate'] = 1

print("Training Set\n")
print("Original train_dataset:\n" + str(train["root_label"][0:20]))
print("\nBinarized train_dataset:\n" + str(y_train_encoded[0:20]))
print("\nTest Set\n")
print("Original test_dataset:\n" + str(test["root_label"][0:20]))
print("\nBinarized test_dataset:\n" + str(y_test_encoded[0:20]))

Training Set

Original train_dataset:
894      sports
1579    climate
89       sports
165      sports
336      sports
1078    climate
1279    climate
2021    climate
804      sports
633      sports
1213    climate
221      sports
751      sports
1246    climate
34       sports
1820    climate
1048    climate
1265    climate
1300    climate
919      sports
Name: root_label, dtype: object

Binarized train_dataset:
894     0
1579    1
89      0
165     0
336     0
1078    1
1279    1
2021    1
804     0
633     0
1213    1
221     0
751     0
1246    1
34      0
1820    1
1048    1
1265    1
1300    1
919     0
Name: root_label, dtype: object

Test Set

Original test_dataset:
742      sports
889      sports
60       sports
446      sports
968      sports
404      sports
808      sports
288      sports
2006    climate
1160    climate
260      sports
484      sports
224      sports
1243    climate
120      sports
1221    climate
631      sports
1411    climate
240      sports
1551    climat

In [27]:
wnl = nltk.wordnet.WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(list_word):
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]


In [28]:
analyzer = CountVectorizer().build_analyzer()
stemmer = PorterStemmer()
def stemmed (doc):
    return (stemmer.stem(word) for word in analyzer(doc) if not word.isdigit())

def lemma(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if not word.isdigit())

def nolemma(doc):
    return (word for word in analyzer(doc) if not word.isdigit())

In [29]:
vectorizer_df3_lemma = CountVectorizer(min_df=3, 
                             analyzer=lemma, 
                             stop_words='english')
vectorizer_df5_lemma = CountVectorizer(min_df=5, 
                             analyzer=lemma, 
                             stop_words='english')
vectorizer_df3_nolemma = CountVectorizer(min_df=3, 
                             analyzer=nolemma, 
                             stop_words='english')
vectorizer_df5_nolemma = CountVectorizer(min_df=5, 
                             analyzer=nolemma, 
                             stop_words='english')
vectorizer_df3_stem = CountVectorizer(min_df=3, 
                             analyzer=stemmed, 
                             stop_words='english')
vectorizer_df5_stem = CountVectorizer(min_df=5, 
                             analyzer=stemmed, 
                             stop_words='english')


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
import joblib
from joblib import Memory

# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline0 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('clf', SVC(kernel='linear', C=1)),
],
memory=memory
)

param_grid0 = [
    {
        'vect': [vectorizer_df3_stem, 
                 vectorizer_df5_stem,
                 vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=5, random_state=42),
                       NMF(n_components=5, init='random', random_state=42, max_iter=10000)],
        'clf': [SVC(kernel='linear', C=1.0,max_iter=10000), 
                LogisticRegression(penalty='l1', C=10,max_iter=10000, solver='liblinear'), 
                LogisticRegression(penalty='l2', C=100,max_iter=10000, solver = 'liblinear'),
                GaussianNB()]
    }
]
grid0 = GridSearchCV(pipeline0, cv=5, n_jobs=1, param_grid=param_grid0, scoring='accuracy')
grid0.fit(train['full_text'], y_train_encoded.astype(str).astype(int))
rmtree(location)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function stemmed at 0x7f8461748700>, min_df=3,
                stop_words='english'), 
186     (CNN) Indian police have arrested three Kashmi...
212     India's Mayank Agarwal , left, talks with Axar...
495     Game recap\n\nOffensive MVP\n\nTanner Morgan, ...
190     Kashmiri students assaulted and bowler Mohamme...
890     Lincoln High School’s marching band was named ...
                              ...                        
303     Seacrest out, Ted Lasso in. Ryan Seacrest and ...
1012    Citrusw00d Productions unveils films minted as...
267     FILE - Tennessee Titans running back Derrick H...
1763    A swarm of earthquakes has been rattling the o...
1588    A very strong 5.5 earthquake was just recorded...
Name: full_text, Length: 1325, dtype: object, 
186     0
212     0
495     0
190     0
890   

In [18]:
gg0 = pd.DataFrame(grid0.cv_results_)

In [19]:
gg00 = gg0.sort_values(by='mean_test_score',ascending=False)
gg00

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
25,0.075437,0.000911,2.002199,0.092823,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.918675,0.936747,0.915408,0.924471,0.909366,0.920933,0.009296,1
13,0.075936,0.000732,1.997829,0.095103,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.921687,0.933735,0.915408,0.924471,0.909366,0.920933,0.008262,1
17,0.093485,0.001213,0.083722,0.003663,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.912651,0.933735,0.912387,0.915408,0.918429,0.918522,0.007915,3
15,0.095489,0.001412,0.084397,0.003628,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.912651,0.933735,0.912387,0.915408,0.915408,0.917918,0.008014,4
29,0.091587,0.001193,0.083836,0.003853,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.912651,0.933735,0.912387,0.915408,0.912387,0.917313,0.008290,5
26,0.086722,0.001053,4.670132,0.223706,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.900602,0.927711,0.915408,0.912387,0.927492,0.916720,0.010170,6
14,0.085103,0.001052,4.638866,0.209409,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.900602,0.930723,0.915408,0.912387,0.924471,0.916718,0.010358,7
21,0.096783,0.001995,0.089148,0.003090,"LogisticRegression(C=10, max_iter=10000, penal...","NMF(init='random', max_iter=10000, n_component...",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.900602,0.933735,0.918429,0.912387,0.918429,0.916716,0.010714,8
24,0.078035,0.000794,2.001578,0.094888,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.897590,0.939759,0.915408,0.915408,0.915408,0.916715,0.013431,9
12,0.079151,0.000962,1.998965,0.096379,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=5, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.900602,0.939759,0.912387,0.915408,0.915408,0.916713,0.012750,10


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
import joblib
from joblib import Memory

# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline1 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('clf', SVC(kernel='linear', C=1)),
],
memory=memory
)

param_grid1 = [
    {
        'vect': [vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=50, random_state=42),
                       NMF(n_components=50, init='random', random_state=42, max_iter=10000)],
        'clf': [SVC(kernel='linear', C=1.0,max_iter=10000), 
                LogisticRegression(penalty='l1', C=10,max_iter=10000, solver='liblinear'), 
                LogisticRegression(penalty='l2', C=100,max_iter=10000, solver = 'liblinear'),
                GaussianNB()]
    }
]
grid1 = GridSearchCV(pipeline1, cv=5, n_jobs=1, param_grid=param_grid1, scoring='accuracy')
grid1.fit(train['full_text'], y_train_encoded.astype(str).astype(int))
rmtree(location)

In [ ]:
    
print(grid1.best_params_)


In [ ]:
y_train_encoded.astype(str).astype(int)

In [ ]:
gg1 = pd.DataFrame(grid1.cv_results_)

In [ ]:
gg2 = gg1.sort_values(by='mean_test_score',ascending=False)
gg2

In [ ]:
gg2.shape

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
import joblib
from joblib import Memory

# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline2 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('clf', SVC(kernel='linear', C=1)),
],
memory=memory
)

param_grid2 = [
    {
        'vect': [vectorizer_df3_stem, 
                 vectorizer_df5_stem, 
                 vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        
        'reduce_dim': [TruncatedSVD(n_components=500, random_state=42), 
                       NMF(n_components=500, init='random', random_state=42, max_iter=10000)],
        
        'clf': [SVC(kernel='linear', C=1.0,max_iter=10000), 
                LogisticRegression(penalty='l1', C=10,max_iter=10000, solver='liblinear'), 
                LogisticRegression(penalty='l2', C=100,max_iter=10000, solver = 'liblinear'),
                GaussianNB()]
    }
]
grid2 = GridSearchCV(pipeline2, cv=5, n_jobs=1, param_grid=param_grid2, scoring='accuracy')
grid2.fit(, y_train_encoded.astype(str).astype(int))
rmtree(location)

In [ ]:
gg3 = pd.DataFrame(grid2.cv_results_)

gg4 = gg3.sort_values(by='mean_test_score',ascending=False)


In [ ]:
gg4.shape

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
import joblib
from joblib import Memory

# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline2 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('clf', SVC(kernel='linear', C=1)),
],
memory=memory
)

param_grid2 = [
    {
        'vect': [vectorizer_df3_stem, 
                 vectorizer_df5_stem, 
                 vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        
        'reduce_dim': [TruncatedSVD(n_components=500, random_state=42), 
                       NMF(n_components=500, init='random', random_state=42, max_iter=10000)],
        
        'clf': [SVC(kernel='linear', C=1.0,max_iter=10000), 
                LogisticRegression(penalty='l1', C=10,max_iter=10000, solver='liblinear'), 
                LogisticRegression(penalty='l2', C=100,max_iter=10000, solver = 'liblinear'),
                GaussianNB()]
    }
]
grid2 = GridSearchCV(pipeline2, cv=5, n_jobs=1, param_grid=param_grid2, scoring='accuracy')
grid2.fit(train_clean, y_train_encoded.astype(str).astype(int))
rmtree(location)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function stemmed at 0x7feb980d0b80>, min_df=3,
                stop_words='english'), 
1308    sunny and highs in the s that s the forecast f...
1230    on nov rep pete stauber voted no on president ...
1691    sacramento was the place to be this past weeke...
1126    griffith university is driving the constructio...
1718    at this year s sema specialty equipment manufa...
                              ...                        
812     introducing martial arts school listings on bl...
358     are the san francisco ers back only time will ...
585     along with the carter kids other celebrity chi...
928     la mini serie que estuvo durante varias semana...
216     england confirmed it will go to australia for ...
Name: full_text, Length: 1325, dtype: object, 
1308    1
1230    1
1691    1
1126    1
1718  

In [33]:
gg3 = pd.DataFrame(grid2.cv_results_)

gg4 = gg3.sort_values(by='mean_test_score',ascending=False)

In [34]:
gg4

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
16,0.158817,0.017307,4.684332,0.450781,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.984940,0.960843,0.984894,0.978852,0.978852,0.977676,0.008843,1
26,0.178048,0.016352,4.737705,0.453073,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.981928,0.969880,0.972810,0.975831,0.987915,0.977673,0.006493,2
29,0.173082,0.004269,0.095920,0.007216,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.987952,0.960843,0.972810,0.975831,0.984894,0.976466,0.009600,3
28,0.161943,0.005134,4.704906,0.471840,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.978916,0.969880,0.972810,0.972810,0.987915,0.976466,0.006438,3
25,0.152860,0.003403,2.046377,0.204721,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.972892,0.966867,0.972810,0.975831,0.990937,0.975867,0.008078,5
27,0.192599,0.016358,0.120809,0.027809,"LogisticRegression(C=100, max_iter=10000, solv...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=100, max_iter=100...",0.984940,0.960843,0.972810,0.975831,0.984894,0.975864,0.008932,6
14,0.182375,0.022581,4.785890,0.517785,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function lemma at 0x...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.984940,0.957831,0.981873,0.978852,0.972810,0.975261,0.009593,7
12,0.183908,0.026981,2.068864,0.218598,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.975904,0.954819,0.984894,0.975831,0.981873,0.974664,0.010520,8
17,0.178633,0.005347,0.095805,0.007750,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function nolemma at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.981928,0.954819,0.981873,0.978852,0.975831,0.974661,0.010173,9
13,0.149733,0.011073,2.032738,0.202493,"LogisticRegression(C=10, max_iter=10000, penal...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function stemmed at ...,"{'clf': LogisticRegression(C=10, max_iter=1000...",0.975904,0.945783,0.978852,0.978852,0.987915,0.973461,0.014416,10
